In [16]:
import joblib
import praw
import data_scrapping as ds
from urllib.parse import urlparse
import re
import pandas as pd
import nltk
import pickle

In [17]:
def get_input(url_to_check):
    
    reddit = praw.Reddit(client_id = '#', \
                                    client_secret = '#',\
                                    user_agent = '#',\
                                    username = '#',\
                                    password = '#')

    post = reddit.submission(url = url_to_check)
    title_c = ds.process_strings(str(post.title))
    content_c = ds.process_strings(str(post.selftext))

    post.comments.replace_more(limit=0)
    comments = ""
    for comment in post.comments.list():
        comments = comments + comment.body

    comments = ds.process_strings(comments)

    comments_c = ds.process_strings(comments)

    url = post.url
    parsed = urlparse(url)
    url_path_c = ds.process_strings(" ".join(re.split('/|-|_', str(parsed.path))))
    
    insert = {
        "title" : [title_c],
        "content" : [content_c],
        "comments" : [comments_c],
        "url_path" : [url_path_c]
    }
    df = pd.DataFrame(insert)
    
    df["merged"] = df["title"] + df["url_path"] + df["comments"] + df["content"]
    
    return df.merged

In [18]:
def fit_to_dict(df_col):
    vec = pickle.load(open("vector.pickle", "rb"))
    trans = vec.transform(df_col)
    return trans

In [19]:
def predict(inp):
    
    frame = get_input(inp)
    changed = fit_to_dict(frame)
    trained_model = joblib.load("flair_predictor.joblib")
    return trained_model.predict(changed)

In [21]:
if __name__ == "__main__":
    i = input()
    print(predict(i))

https://www.reddit.com/r/india/comments/cgr12r/spotted_in_delhi/
['AskIndia']
